# Crypto analyser

### Import library

In [64]:
from datetime import datetime
from binance.client import Client

import os
import pandas as pd

from sqlalchemy import create_engine, inspect

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display
from datetime import datetime, date




### API connection
The historical data regarding crypto-currencies come from www.binance.com.
The API key is required. It is free, and here is how to get it: https://www.binance.com/en/support/faq/360002502072.

In [65]:
binance_api_key = os.environ.get('binance_api_key')
binance_api_secret = os.environ.get('binance_api_secret')
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

### Database connection & functions

In [66]:
MYSQL_HOSTNAME = 'localhost'
MYSQL_USER = os.environ.get('DB_USER')
MYSQL_PASSWORD = os.environ.get('DB_PASS')
MYSQL_DATABASE = 'cranalyserDB'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'

engine = create_engine(connection_string)
inspector = inspect(engine)

In [67]:
def table_exist(table: str) -> bool:
    if inspector.has_table(table_name=table): return True
    else: return False

def save_df(df, table: str):
    df.to_sql(table, engine, if_exists='append')

def get_df(table: str) -> pd.DataFrame:
    df = pd.read_sql_table(table, engine)
    return df



### Downloading date functions

In [68]:
bin_sizes = {"1m": 1, "1h": 60, "1d": 1440}

def update_checker(cr_code, frequency, data):

    """Checks the range of data needed for download.

    Parameters
    -----------
    cr_code : str, crypto-currency code e.g BTCUSDT available in API
    frequency : str, determines the data download interval
    data : pd.DataFrame, currently dataframe existing in database

    Returns
    ----------
    old, new : datatime, points defining the range of data to download"""

    if len(data) > 0: old = data["timestamp"].iloc[-1]
    else: old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    new = pd.to_datetime(binance_client.get_klines(symbol=cr_code, interval=frequency)[-1][0], unit='ms')
    return old, new

def get_from_binance(cr_code, frequency='1h', save_db=False, save_csv=False, return_df=True):
    
    """Downloads data from binance.

    Parameter
    ---------
    cr_code : str, crypto-currency code e.g BTCUSDT available in API
    frequency : str, optional,
        determines the data download interval: 1m, 1h, 1d (default is 1h)
        You can specify available frequency in bin_sizes
    save_db : bool, optional,
        If True new data will be append to database (default is False)
    save_csv : bool, optional,
        If True dataframe will be save to csv (default is False)
    return_df : bool, optional,
        If True dataframe is returned (default is True)

    Returns
    --------
    dataframe (for return_df is True)
    """

    tablename = '%s_%s_data' % (cr_code.lower(), frequency)


    if table_exist(tablename):
        data_df = get_df(tablename)
    else:
        data_df = pd.DataFrame()

    oldest_point, newest_point = update_checker(cr_code, frequency, data_df)
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'):
        print('Downloading all available %s data for %s. Wait...' % (frequency, cr_code))
    else: print('Downloading new available %s data for %s. Wait...!' % (frequency, cr_code))
    data_api = binance_client.get_historical_klines(cr_code, frequency, oldest_point.strftime("%d %b %Y %H:%M:%S"),
                                                    newest_point.strftime("%d %b %Y %H:%M:%S"))
    data_api_df = pd.DataFrame(data_api,
                               columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                                        'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
    data_api_df['timestamp'] = pd.to_datetime(data_api_df['timestamp'], unit='ms')
    #data_api_df['timestamp'] = data_api_df['timestamp'].apply(lambda x: )
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data_api_df)
        data_df = data_df[:len(data_df)-1]
        data_df = data_df.append(temp_df)
    else: data_df = data_api_df
    data_df.set_index('timestamp', inplace=True)
    if save_db:
        save_df(data_df, tablename)
    if save_csv:
        data_df.to_csv(tablename)
    print('All data is ready!')
    if return_df:
        return data_df

In [69]:
binance_symbols = ["BTCUSDT"]
for symbol in binance_symbols:
    get_from_binance(symbol, '1d', save_db=True, return_df=True)

All data is ready!


In [70]:
df = get_df('btcusdt_1d_data')
df

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-08-17,4261.48000000,4485.39000000,4200.74000000,4285.08000000,795.15037700,1503014399999,3454770.05073206,3427,616.24854100,2678216.40060401,8733.91139481
1,2017-08-18,4285.08000000,4371.52000000,3938.77000000,4108.37000000,1199.88826400,1503100799999,5086958.30617151,5233,972.86871000,4129123.31651808,9384.14140858
2,2017-08-19,4108.37000000,4184.69000000,3850.00000000,4139.98000000,381.30976300,1503187199999,1549483.73542151,2153,274.33604200,1118001.87008735,9184.08552906
3,2017-08-20,4120.98000000,4211.08000000,4032.62000000,4086.29000000,467.08302200,1503273599999,1930364.39032646,2321,376.79594700,1557401.33373730,10125.41408414
4,2017-08-21,4069.13000000,4119.62000000,3911.79000000,4016.00000000,691.74306000,1503359999999,2797231.71402728,3972,557.35610700,2255662.55315837,11706.76997007
...,...,...,...,...,...,...,...,...,...,...,...,...
2639,2021-03-26,51293.78000000,55073.46000000,51214.60000000,55025.59000000,63813.77469200,1616803199999,3397036140.75163746,1773150,31822.54523800,1694326043.33711574,0
2640,2021-03-27,55025.59000000,56700.36000000,53950.00000000,55817.14000000,50105.47505500,1616889599999,2767070175.86193082,1528776,24739.94847100,1365965317.25052359,0
2641,2021-03-28,55817.14000000,56559.75000000,54691.84000000,55777.63000000,39050.38751100,1616975999999,2178777329.44521665,1551283,19262.72593500,1074959545.95900884,0
2642,2021-03-29,55777.65000000,58405.82000000,54800.01000000,57635.47000000,67857.93739800,1617062399999,3877131109.73464578,1983437,34373.80455600,1963800173.65605960,0


### Selecting data - functions & create widgets

In [99]:
output = widgets.Output()

cryptocurrencies = ['btcusdt']

dropdown_currency = widgets.Dropdown(options=cryptocurrencies, description='Select currency:')
interval_radio = widgets.RadioButtons(options=['1m', '1h', '1d'], value='1d', description='Data interval:')

dates = pd.date_range(datetime(2017, 8, 17), date.today().strftime("%Y %m %d"), freq='D')
options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

date_range_slider = widgets.SelectionRangeSlider(options=options, index=index,
                                          description='Dates', orientation='horizontal', layout={'width': '500px'})
get_data_button = widgets.Button(description='Get data!')


def get_output(currency, interval, start, stop):
    output.clear_output()

    data_fr = get_df(f'{currency.lower()}_{interval}_data')
    data_fr = data_fr[(data_fr['timestamp'] >= start) & (data_fr['timestamp'] <= stop)]

    with output:
        display(data_fr)

def button_event_handler(button):
    start = date_range_slider.value[0]
    stop = date_range_slider.value[1]
    get_output(dropdown_currency.value, interval_radio.value, start, stop)

get_data_button.on_click(button_event_handler)



### Selecting data - widgets

In [100]:
display(dropdown_currency,interval_radio,date_range_slider, get_data_button)


Dropdown(description='Select currency:', options=('btcusdt',), value='btcusdt')

RadioButtons(description='Data interval:', index=2, options=('1m', '1h', '1d'), value='1d')

SelectionRangeSlider(description='Dates', index=(0, 1321), layout=Layout(width='500px'), options=((' 17 Aug 20…

Button(description='Get data!', style=ButtonStyle())

In [101]:
display(output)


Output()